Stanislas Deneuville - Emmanuel Ferrandi - Pol Grisart - Marine Médard
# Project of data science :  Face recognition in a video and counting
16/11/2018

## Part II : Train&Use

In [1]:
import random

__split__ : split the set into two well mixed set 


In : 
* data : list of all the images 

Out : 
* train_set : list of the images in the training set representing 66% of data
* test_set : list of the images in the test set representing the other 34% of data

In [ ]:
TRAIN_TEST_RATIO = 0.66
# TODO equlize data, have the same number of each value in both categories to avoid bias (another function?)
def split(data:list, train_test_ratio:int=TRAIN_TEST_RATIO, random_split=True) :
    if random_split:
        # Shuffle
        random.shuffle(data)
    
    # Split data
    cut_index = round(len(data) * train_test_ratio) 
    train_set = data[:cut_index]
    test_set = data[cut_index:]
    return(train_set, test_set)

__train_neuural_network__ : function that creates a neural network and trains it with the train_set 

In [ ]:
def train_neural_network(train_set):

__evaluate_performance__ : function that applys the neural network on the images in the test_set and compare with the real number of faces in these images 

In [ ]:
def evaluate_performance() :